# Word2Vec

Word2Vec是一种基于神经网络的embedding模型，由谷歌的研究人员Tomas Mikolov等在2013年提出。Word2Vec的作者认为之前的文本向量化方法很少考虑词与词之间的意义关联，所以效果不佳。因此，Word2Vec将单词映射到低维向量空间中，使得相似词在向量空间中也保持相近的距离。

<img src="images/img_3.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 500px;">

<img src="images/img_7.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 500px;">

Word2Vec主要由两种模型架构组成：**Skip-Gram**和**Continuous Bag of Words（CBOW）**。

Skip-Gram通过当前词预测其上下文词。给定一个中心词，模型会预测一个固定长度的上下文窗口（前后若干词）中的所有词。训练过程中，模型会学习到每个词的向量表示，使得能够更好地预测这些上下文词。与Skip-Gram相反，CBOW的目标是通过上下文词预测当前词。给定一个上下文窗口中的所有词，模型会预测这个窗口的中心词。

Word2Vec模型的核心思想是通过训练神经网络，使得单词与其上下文之间的关系可以在向量空间中被有效地表示。Word2Vec的输入层是一个one-hot向量（one-hot vector），长度为词汇表大小（$V$）。紧接着是一个投影层，由输入层经过一个权重矩阵 $W$（维度为$V \times N$，$N$为嵌入向量的维度），投影到 $N$ 维向量空间中。投影层的输出通过另一个权重矩阵 $W'$（维度为$N \times V$），映射回一个词汇表大小的向量，此为输出层。最后经过一个Softmax层得到每个词的概率。在训练过程中，Word2Vec模型通过最大化目标函数（取决于使用Skip-Gram还是CBOW，以及采用的优化函数的方法）来更新权重矩阵，从而使得embedding能够捕捉到词汇的语义信息。

> 从监督学习的角度来说，word2vec 本质上是一个基于神经网络的多分类问题，当输出词语非常多时， 我们则需要一些像层级 Softmax 和负采样之类的 trick 来加速训练。但从自然语言处理的角度来说， word2vec 关注的并不是神经网络模型本身，而是训练之后得到的词汇的向量化表征。 这种表征使得最后的词向量维度要远远小于词汇表大小，所以 word2vec 从本质上来说是一种降维操作。 我们把数以万计的词汇从高维空间中降维到低维空间中，大大方便了后续的 NLP 分析任务。


## 优缺点

优点：
● 由于采用了浅层神经网络，训练速度较快，适合处理大规模数据。
● 能够捕捉到词与词之间的语义关系，如词性、同义词等。
● 可以用于多种自然语言处理任务，如文本分类、情感分析、机器翻译等。

缺点：
● 每个词只有一个固定的向量表示，无法处理多义词的不同语义。
● 无法考虑词在不同上下文中的语义变化。

## 应用场景
● 信息检索
● 文本分类
● 推荐系统，如根据用户的评论调整推荐的产品



## CBOW
CBOW 模型的应用场景是要根据上下文预测中间的词，所以输入便是上下文词，当然原始的单词是无法作为输入的， 这里的输入仍然是每个词汇的 One-Hot 向量，输出 Y 为给定词汇表中每个词作为目标词的概率。

CBOW 模型结构是一种普通的神经网络结构。主要包括输入层、中间隐藏层、最后的输出层。 以输入、输出样本 $(Context(w),w)$ 为例对 CBOW 模型的三个网络层进行简单说明， 其中假设 Context(w)由 $w$ 前后各 $c$个词构成。
  - **输入层**: 包含 $Context(w)$ 中 $2c$ 个词的词向量 $v(Context(w)1),v(Context(w)2),⋯,v(Context(w)2c)\in{R^m}$。 这里， $m$ 的含义同上表示词向量的长度
  - **投影层**: 将输入层的 $2c$ 个向量做求和累加，即 $x_w=\sum_{i=1}^{2c}v(Context(w)_i)\in{R^m}$
  - **输出层**: 输出层对应一颗二叉树，它是以语料中出现过的词当叶子节点，以各词在语料中出现的次数当权值构造出来的 Huffman 树。在这棵 Huffman 树中， 叶子节点共 $N(=|D|)$ 个，分别对应词典 $D$ 中的词，非叶子节点 $N−1$ 个。

普通的基于神经网络的语言模型输出层一般就是利用 softmax 函数进行归一化计算，这种直接 softmax 的做法主要问题在于计算速度， 尤其是我们采用了一个较大的词汇表的时候，对大的词汇表做求和运算，softmax 的分运算会非常慢，直接影响到了模型性能。

可以看到，上面提到的取消隐藏层，投影层求和平均都可以一定程度上减少计算量，但输出层的数量在那里， 比如语料库有 500W 个词，那么隐藏层就要对 500W 个神经元进行全连接计算，这依然需要庞大的计算量。 word2vec 算法又在这里进行了训练优化.

除了层级 softmax 输出之外，还有一种叫做负采样的训练 trick

目标函数：
$$
J=\sum_{w \in corpus} P(w|context(w)
$$

CBOW 在 NNLM 基础上有以下几点创新

1. 取消了隐藏层，减少了计算量
2. 采用上下文划窗而不是前文划窗，即用上下文的词来预测当前词
3. 投影层不再使用各向量拼接的方式，而是简单的求和平均

<img src="images/img_4.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 400px;">

## Skip-Gram

Skip-gram 模型的应用场景是要根据中间词预测上下文词，所以输入 $X$ 是任意单词， 输出 $Y$为给定词汇表中每个词作为上下文词的概率。

Skip-gram 模型与 CBOW 模型翻转，也是也是一种普通的神经网络结构， 同样也包括输入层、中间隐藏层和最后的输出层。继续以输入输出样本 $(Context(w)，w)$
 为例 对 Skip-gram 模型的三个网络层进行简单说明， 其中假设 $Context(w)$ 由 $w$ 前后各 $c$ 个词构成。数学细节如下:
- **输入层**: 只含当前样本的中心词 $w$ 的词向量 $v(w)∈Rm$
- **投影层**: 这是个恒等投影，把 $v(w)$ 投影到 $v(w)$，因此，这个投影层其实是多余的。 这里之所以保留投影层主要是方便和 CBOW 模型的网络结构做对比
- **输出层**: 和 CBOW 模型一样，输出层也是一棵 Huffman 树

<img src="images/img_5.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 400px;">

<img src="images/img_6.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 400px;">

## 学习资源
● [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)
● [word2vec Parameter Learning Explained](https://arxiv.org/abs/1411.2738)
● [可视化 word2vec](https://ronxin.github.io/wevi/)
  [万物皆可Vector之语言模型：从N-Gram到NNLM、RNNLM](https://mp.weixin.qq.com/s/XiHzsjTK0TpQSjaOC6qFjw)
● [万物皆可Vector之Word2vec：2个模型、2个优化及实战使用](https://mp.weixin.qq.com/s/hoQXBo2r4WGgxIGtODZkFw)
